# A5 小書產生器（v17）— 每段自動換頁可切換

In [ ]:
import io, re, base64, urllib.request, time
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from docx import Document
from docx.shared import Mm, Pt, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from PIL import Image
FWP='％'
def ensure_rgb_jpeg(img_bytes, quality=90):
    from PIL import Image, ImageFile
    ImageFile.LOAD_TRUNCATED_IMAGES = True
    im = Image.open(io.BytesIO(img_bytes))
    if im.mode not in ('RGB','L'): im = im.convert('RGB')
    out = io.BytesIO(); im.save(out, format='JPEG', quality=quality); return out.getvalue()
def parse_and_strip_size_anywhere(text, default_pt=12):
    import re
    m = re.search(r'\[(\d+)pt\]', text)
    if m: size=int(m.group(1)); return size, text[:m.start()]+text[m.end():]
    return default_pt, text
def has_skip_image_tag(text):
    import re
    return bool(re.search(r'\[\s*無圖\s*\]\s*$', text))
def strip_skip_image_tag(text):
    import re
    return re.sub(r'\[\s*無圖\s*\]\s*$', '', text)
def tokenize_line(line):
    import re
    tokens=[]; i=0
    pattern=re.compile(r"\[\s*左圖\s*(\d+)?\s*[%"+FWP+"]?\s*\]\s*\[\s*右圖\s*(\d+)?\s*[%"+FWP+"]?\s*\]|\[\s*圖片\s*(\d+)?\s*[%"+FWP+"]?\s*\]")
    while True:
        m=pattern.search(line,i)
        if not m:
            tokens.append(('text', line[i:])); break
        if m.start()>i: tokens.append(('text', line[i:m.start()]))
        if m.group(1) is not None or m.group(2) is not None:
            lp=int(m.group(1)) if m.group(1) else None; rp=int(m.group(2)) if m.group(2) else None
            tokens.append(('double', lp, rp))
        else:
            sp=m.group(3); tokens.append(('single', int(sp) if sp else None))
        i=m.end()
    return tokens
def set_section_to_a5(section):
    section.page_width=Mm(148); section.page_height=Mm(210)
    section.left_margin=Mm(15); section.right_margin=Mm(15)
    section.top_margin=Mm(15); section.bottom_margin=Mm(15)
def add_text_paragraph(doc, text, size_pt, center=False):
    text=text.strip()
    if not text: return None
    p=doc.add_paragraph(text); p.alignment=WD_ALIGN_PARAGRAPH.CENTER if center else WD_ALIGN_PARAGRAPH.LEFT
    for r in p.runs: r.font.size=Pt(size_pt)
    return p
def add_single_image_paragraph(doc, img_bytes, width_pct=None, center=True):
    page_w_mm=148-15-15; page_w_in=page_w_mm/25.4
    if width_pct is None: width_in=page_w_in
    else:
        width_in=page_w_in*(width_pct/100.0); width_in=max(1.0, min(page_w_in, width_in))
    safe=ensure_rgb_jpeg(img_bytes); p=doc.add_paragraph(); p.alignment=WD_ALIGN_PARAGRAPH.CENTER if center else WD_ALIGN_PARAGRAPH.LEFT
    p.add_run().add_picture(io.BytesIO(safe), width=Inches(width_in))
def add_double_image_table(doc, left_bytes, right_bytes, l_pct, r_pct):
    page_w_mm=148-15-15; page_w_in=page_w_mm/25.4; padding=0.2
    base_in=(page_w_in-padding)/2.0
    l_in=base_in if l_pct is None else base_in*(l_pct/100.0)
    r_in=base_in if r_pct is None else base_in*(r_pct/100.0)
    l_in=max(0.8, min(base_in, l_in)); r_in=max(0.8, min(base_in, r_in))
    table=doc.add_table(rows=1, cols=2); table.autofit=True
    for ci,b,w in [(0,left_bytes,l_in),(1,right_bytes,r_in)]:
        cell=table.rows[0].cells[ci]; para=cell.paragraphs[0]; para.alignment=WD_ALIGN_PARAGRAPH.JUSTIFY
        para.add_run().add_picture(io.BytesIO(ensure_rgb_jpeg(b)), width=Inches(w))
def add_page_number_footer(section):
    footer=section.footer; p=footer.paragraphs[0] if footer.paragraphs else footer.add_paragraph(); p.alignment=WD_ALIGN_PARAGRAPH.CENTER
    p.add_run('-'); fld_begin=OxmlElement('w:fldChar'); fld_begin.set(qn('w:fldCharType'),'begin')
    instr=OxmlElement('w:instrText'); instr.text=' PAGE '
    fld_end=OxmlElement('w:fldChar'); fld_end.set(qn('w:fldCharType'),'end')
    r=OxmlElement('w:r'); r.append(fld_begin); r.append(instr); r.append(fld_end); p._p.append(r); p.add_run('-')
def make_data_uri_download(data: bytes, filename: str) -> HTML:
    b64=base64.b64encode(data).decode()
    return HTML(f'<a download="{filename}" href="data:application/vnd.openxmlformats-officedocument.wordprocessingml.document;base64,{b64}">📥 點此下載 {filename}</a>')
def image_manager(title: str):
    uploader = widgets.FileUpload(accept='image/*', multiple=True, description=f'上傳{title}')
    lst = widgets.Select(options=[], rows=6, description='順序')
    btn_up = widgets.Button(description='上移'); btn_down = widgets.Button(description='下移')
    btn_del = widgets.Button(description='刪除'); btn_clear = widgets.Button(description='清空')
    add_box = widgets.Textarea(placeholder='貼入圖片 URL 或 data:image/...;base64,... 或純 base64（每行一張）', layout=widgets.Layout(height='70px', width='100%'))
    add_btn = widgets.Button(description='新增到清單', button_style='info')
    status = widgets.HTML('')
    images = []
    def refresh():
        lst.options=[f"{i+1:02d}. {it['name']} ({len(it['bytes'])//1024}KB)" for i,it in enumerate(images)]; lst.index=0 if images else None
    def on_upload(change):
        nonlocal images
        for meta in uploader.value:
            content=meta.get('content', b''); name=meta.get('name','image')
            if content: images.append({'name':name, 'bytes':content})
        uploader.value.clear(); refresh(); status.value=f"<span style='color:green'>已加入 {len(images)} 張</span>"
    uploader.observe(on_upload, names='value')
    def move(delta):
        if not images or lst.index is None: return
        i=lst.index; j=i+delta
        if 0<=j<len(images): images[i],images[j]=images[j],images[i]; refresh(); lst.index=j
    btn_up.on_click(lambda _: move(-1)); btn_down.on_click(lambda _: move(1))
    btn_del.on_click(lambda _: (images.pop(lst.index), refresh()) if (images and lst.index is not None) else None)
    btn_clear.on_click(lambda _: (images.clear(), refresh()))
    def do_add_from_text(_):
        nonlocal images
        lines=[ln.strip() for ln in add_box.value.splitlines() if ln.strip()]; added=0
        for ln in lines:
            try:
                if ln.startswith('data:image'): b64=ln.split(',',1)[1]; b=base64.b64decode(b64)
                elif ln.startswith('http://') or ln.startswith('https://'):
                    with urllib.request.urlopen(ln) as resp: b=resp.read()
                else:
                    b=base64.b64decode(ln, validate=True)
                images.append({'name':f'added_{len(images)+1}.jpg','bytes':b}); added+=1
            except Exception:
                pass
        add_box.value=''; refresh(); status.value=f"<span style='color:green'>新增 {added} 張</span>" if added else "<span style='color:#a00'>沒有成功新增</span>"
    add_btn.on_click(do_add_from_text)
    ui = widgets.VBox([uploader, widgets.HBox([lst, widgets.VBox([btn_up, btn_down, btn_del, btn_clear])]), widgets.HBox([add_box, widgets.VBox([add_btn])]), status])
    return ui, images
txt_cover = widgets.Textarea(value='書名或口號[18pt]\n[圖片50%]\n種是希望', description='封面文字', layout=widgets.Layout(width='100%', height='150px'))
txt_body  = widgets.Textarea(value='第一段文字。\n\n第二段文字。\n\n第三段文字。', description='內頁文字', layout=widgets.Layout(width='100%', height='200px'))
txt_back  = widgets.Textarea(value='封底簡介……\n[左圖60%][右圖60%]', description='封底文字', layout=widgets.Layout(width='100%', height='150px'))
cover_default_pct = widgets.BoundedIntText(value=100, min=10, max=100, step=1, description='封面預設%')
body_default_pct  = widgets.BoundedIntText(value=100, min=10, max=100, step=1, description='內頁預設%')
back_default_pct  = widgets.BoundedIntText(value=100, min=10, max=100, step=1, description='封底預設%')
chk_pagenum = widgets.Checkbox(value=True, description='內頁加頁碼 (-1-)')
chk_pagebreak = widgets.Checkbox(value=False, description='每段文字自動換頁')
ui_cover, imgs_cover = image_manager('封面圖片（可多張）')
ui_body,  imgs_body  = image_manager('內頁圖片（可多張）')
ui_back,  imgs_back  = image_manager('封底圖片（可多張）')
btn = widgets.Button(description='生成 A5 小書（docx）', button_style='success')
out = widgets.Output()
display(widgets.VBox([
    widgets.HBox([widgets.VBox([txt_cover, cover_default_pct]), widgets.VBox([chk_pagenum, chk_pagebreak])]),
    ui_cover,
    widgets.Label('—— 內頁 ——'),
    txt_body,
    body_default_pct,
    ui_body,
    widgets.Label('—— 封底 ——'),
    txt_back,
    back_default_pct,
    ui_back,
    btn,
    out
]))
def emit_stream(doc, text, size_pt, imgs, default_pct, center=False):
    skip_img = has_skip_image_tag(text); text = strip_skip_image_tag(text)
    tokens = tokenize_line(text)
    for tk in tokens:
        if tk[0]=='text':
            add_text_paragraph(doc, tk[1], size_pt, center=center)
        elif tk[0]=='single' and (not skip_img) and imgs:
            pct = tk[1] if tk[1] is not None else (default_pct or 100)
            add_single_image_paragraph(doc, imgs.pop(0), width_pct=pct, center=True)
        elif tk[0]=='double' and (not skip_img) and len(imgs)>=2:
            lp, rp = tk[1], tk[2]
            add_double_image_table(doc, imgs.pop(0), imgs.pop(0), lp, rp)
def build_docx_bytes():
    cover_text = txt_cover.value; body_text = txt_body.value; back_text = txt_back.value
    cover_imgs = [it['bytes'] for it in imgs_cover]
    body_imgs  = [it['bytes'] for it in imgs_body]
    back_imgs  = [it['bytes'] for it in imgs_back]
    cover_pct=int(cover_default_pct.value or 100); body_pct=int(body_default_pct.value or 100); back_pct=int(back_default_pct.value or 100)
    doc = Document(); set_section_to_a5(doc.sections[0])
    cover_size, cover_text2 = parse_and_strip_size_anywhere(cover_text, default_pt=18)
    emit_stream(doc, cover_text2, cover_size, cover_imgs, cover_pct, center=True)
    if cover_imgs: add_single_image_paragraph(doc, cover_imgs.pop(0), width_pct=cover_pct, center=True)
    doc.add_section(); set_section_to_a5(doc.sections[-1])
    # 封底不繼承頁腳，且清空
    doc.sections[-1].footer.is_linked_to_previous = False
    _ = [setattr(p, 'text','') for p in doc.sections[-1].footer.paragraphs]
    # 封底不繼承頁腳，且清空
    doc.sections[-1].footer.is_linked_to_previous = False
    _ = [setattr(p, 'text','') for p in doc.sections[-1].footer.paragraphs]
    # 內文從本節開始，不繼承上一節的頁腳（確保封面無頁碼）
    doc.sections[-1].footer.is_linked_to_previous = False
    if chk_pagenum.value: add_page_number_footer(doc.sections[-1])
    raw_lines = body_text.split('\n')
    non_empty = [ln for ln in raw_lines if ln.strip()!='']
    total = len(non_empty); done = 0
    for raw in raw_lines:
        line = raw.strip()
        if line=='': doc.add_paragraph(''); continue
        size_pt, text = parse_and_strip_size_anywhere(line, default_pt=12)
        tokens = tokenize_line(text)
        has_tag = any(tk[0] != 'text' for tk in tokens)
        emit_stream(doc, text, size_pt, body_imgs, body_pct, center=False)
        if (not has_tag) and body_imgs:
            add_single_image_paragraph(doc, body_imgs.pop(0), width_pct=body_pct, center=True)
        done += 1
        if chk_pagebreak.value and done < total:
            doc.add_page_break()
    doc.add_section(); set_section_to_a5(doc.sections[-1])
    # 封底：不繼承頁首頁尾並清空，確保無頁碼
    doc.sections[-1].footer.is_linked_to_previous = False
    doc.sections[-1].header.is_linked_to_previous = False
    for p in list(doc.sections[-1].footer.paragraphs):
        p._element.getparent().remove(p._element)
    doc.sections[-1].footer.add_paragraph('')
    for p in list(doc.sections[-1].header.paragraphs):
        p._element.getparent().remove(p._element)
    doc.sections[-1].header.add_paragraph('')
    back_size, back_text2 = parse_and_strip_size_anywhere(back_text, default_pt=12)
    emit_stream(doc, back_text2, back_size, back_imgs, back_pct, center=True)
    if back_imgs: add_single_image_paragraph(doc, back_imgs.pop(0), width_pct=back_pct, center=True)
    buf = io.BytesIO(); doc.save(buf); data = buf.getvalue()
    _ = Document(io.BytesIO(data))
    return data
def on_click(_):
    with out:
        clear_output();
        try:
            data = build_docx_bytes(); fname = f'a5_book_{int(time.time())}.docx'
            display(HTML('<div style="color:green;">完成！</div>'))
            b64=base64.b64encode(data).decode()
            display(HTML(f'<a download="{fname}" href="data:application/vnd.openxmlformats-officedocument.wordprocessingml.document;base64,{b64}">📥 點此下載 {fname}</a>'))
        except Exception as e:
            display(HTML(f'<div style="color:red;">失敗：{e}</div>'))
btn.on_click(on_click)


In [ ]:
import os, hashlib, shutil
import ipywidgets as widgets
from pathlib import Path

# 緩存設定
CACHE_DIR = Path('./.cache_a5app')
CACHE_DIR.mkdir(exist_ok=True)
p_lite_mode = widgets.Checkbox(value=True, description='輕量模式（縮圖/壓縮，適合手機下載）')
display(p_lite_mode)

def _hash_key(s: str) -> str:
    return hashlib.md5(s.encode('utf-8')).hexdigest()

def cache_put_bytes(key: str, data: bytes) -> Path:
    p = CACHE_DIR / key
    with open(p, 'wb') as f: f.write(data)
    return p

def cache_get_path(key: str) -> Path:
    p = CACHE_DIR / key
    return p if p.exists() else None

# 針對 TTS：包一層快取
def synth_speech_to_mp3_cached(text: str, out_path: str, resolve_params_func=None):
    if resolve_params_func is None:
        # 向後相容：如果你的環境已有 resolve_voice_params()
        resolve_params_func = globals().get('resolve_voice_params')
    params = resolve_params_func() if resolve_params_func else {'engine':'gtts','lang':'zh-TW','tld':'com.tw','slow':False}
    sig = f"{params}|{text}"
    key = _hash_key(sig) + '.mp3'
    cached = cache_get_path(key)
    if cached:
        # 已有緩存，直接複製
        shutil.copyfile(cached, out_path)
        return out_path
    # 生成音檔（沿用你現有的 synth_speech_to_mp3 或內建簡易版）
    if 'synth_speech_to_mp3' in globals():
        synth_speech_to_mp3(text, out_path)
    else:
        # Fallback: 最簡單 gTTS
        from gtts import gTTS
        gTTS(text=text, lang=params.get('lang','zh-TW'), tld=params.get('tld','com.tw'), slow=params.get('slow',False)).save(out_path)
    # 放入快取
    with open(out_path, 'rb') as f:
        cache_put_bytes(key, f.read())
    return out_path

# 影像壓縮工具（Pillow）
def compress_image_for_mobile(img_pil, max_w=1280, quality=72):
    if not p_lite_mode.value:
        return img_pil
    w, h = img_pil.size
    if w > max_w:
        nh = int(h * max_w / w)
        img_pil = img_pil.resize((max_w, nh))
    from io import BytesIO
    buf = BytesIO()
    img_pil.save(buf, format='JPEG', quality=quality, optimize=True)
    buf.seek(0)
    from PIL import Image
    return Image.open(buf)

# 影片參數（輕量）
VIDEO_FPS = 24
VIDEO_BITRATE = '1000k' if p_lite_mode.value else '2500k'

print('✅ 緩存/輕量模式工具就緒：', CACHE_DIR)


In [ ]:
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
def set_page_num_start(section, start=1):
    sectPr = section._sectPr
    pgNumType = sectPr.find(qn('w:pgNumType'))
    if pgNumType is None:
        pgNumType = OxmlElement('w:pgNumType')
        sectPr.append(pgNumType)
    pgNumType.set(qn('w:start'), str(int(start)))

